In [102]:
from arcgis.gis import GIS
from arcgis.geocoding import geocode
from arcgis.mapping import WebMap
from PIL import Image
import arcpy
import math

gis = GIS("home")
aprx = arcpy.mp.ArcGISProject("CURRENT")
OUTPUT_DIR = ("C:/Users/Vincent Mai/Desktop/Projects/GIS/nolli_data")

In [75]:
mapView = aprx.activeView

In [107]:
wgs84 = arcpy.SpatialReference(4326)
web_mercator = arcpy.SpatialReference(3857)

def get_lat_lon(city_name):
    geocode_result = geocode(city_name)[0]
    lat = geocode_result['location']['y']
    lon = geocode_result['location']['x']
    return lat, lon

def get_extent(lat, lon, size):
    point_geo = arcpy.PointGeometry(arcpy.Point(lon, lat), wgs84)
    point_projected = point_geo.projectAs(web_mercator)
    x, y = point_projected.centroid.X, point_projected.centroid.Y
    return arcpy.Extent(x - size, y - size, x + size, y + size)

def crop_image(file, width, height, edge_crop):
    with Image.open(file) as img:
        left = edge_crop
        upper = edge_crop
        right = width - edge_crop
        lower = height - edge_crop
        cropped_img = img.crop((left, upper, right, lower))
        cropped_img.save(file)
        
def move_around(lat, lon, step_size, times):
    locs = []
    n = int(math.sqrt(times))
    for i in range (-n//2, n//2):
        for j in range(-n//2, n//2):
            x = lat + step_size * i
            y = lon + step_size * j
            locs.append((x, y))
    return locs

In [111]:
def get_data(location, prefix=''):
    image_name = prefix + location.split(',')[0]
    width, height = 800, 800
    resolution = 90
    edge_crop = 50
    lat, lon = get_lat_lon(location)
    points = move_around(lat, lon, 0.005, 100)
    for idx, (x, y) in enumerate(points):
        extent = get_extent(x, y, 800)
        mapView.camera.setExtent(extent)
        mapView.exportToPNG(f"{OUTPUT_DIR}/{image_name}_{idx}.png", width, height, resolution)
        crop_image(f"{OUTPUT_DIR}/{image_name}_{idx}.png", width, height, edge_crop)

In [105]:
locations = [
    "New York, New York, USA",
    "Barcelona, Catalonia, Spain",
    "Paris, Île-de-France, France",
    "Amsterdam, North Holland, Netherlands",
    "Venice, Veneto, Italy",
    "Brasília, Federal District, Brazil",
    "Shanghai, China",
    "Amsterdam, North Holland, Netherlands",
    "Venice, Veneto, Italy",
    "Brasília, Federal District, Brazil",
    "Shanghai, China",
    "Hong Kong, China",
    "Moscow, Russia",
    "Singapore",
    "Xi'an, Shaanxi, China",
    "Rome, Lazio, Italy",
    "Dubai, United Arab Emirates",
    "São Paulo, São Paulo, Brazil",
    "Jakarta, Special Capital Region, Indonesia",
    "New Delhi, Delhi, India",
    "Tokyo, Japanb",
    "Vienna, Austria",
    "Kyoto, Japan",
    "Copenhagen, Denmark",
    "Isfahan, Isfahan Province, Iran",
    "Jaipur, Rajasthan, India",
    "Edinburgh, Scotland, UK",
    "Granada, Andalusia, Spain",
    "Montreal, Quebec, Canada",
    "Chengdu, Sichuan, China",
    "St. Petersburg, Russia",
    "Naples, Campania, Italy",
    "Philadelphia, Pennsylvania, USA",
    "Ahmedabad, Gujarat, India"
    "Suzhou, Jiangsu, China",
    "Seoul, South Korea",
    "Lisbon, Portugal",
    "Mexico City, Mexico",
    "Budapest, Hungary",
]

In [114]:
for loc in locations[1:]:
    get_data(loc, prefix='bldg_')